Create Files

In [ ]:
import os

# Define the base directory and the folder names
base_dir = '/content'
folders = ['images1', 'images2', 'images3', 'selectedpi']

# Create the directories if they don't already exist
for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    os.makedirs(folder_path, exist_ok=True)
    print(f"Directory created: {folder_path}")


Delete Files

In [ ]:
import shutil

# Specify the folder path to delete
folder_path = '/content/selectedpi'  # Replace with your folder path

# Delete the folder
try:
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
except FileNotFoundError:
    print(f"Folder '{folder_path}' does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

File Number

In [ ]:
import os

def count_files_in_folder(folder_path):
    file_count = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        file_count += len(filenames)  # Count the number of files in the current directory
    return file_count

# Specify the path to the folder
folder_path1 = '/content/selectedpi'  # Replace with your folder path
folder_path2 = '/content/images2'  # Replace with your folder path
folder_path3 = '/content/images3'  # Replace with your folder path

# Get the number of files in the folder
num_files1 = count_files_in_folder(folder_path1)
num_files2 = count_files_in_folder(folder_path2)
num_files3 = count_files_in_folder(folder_path3)


# Print the result
print(f"Number of files in the folder: {num_files1}")
print(f"Number of files in the folder: {num_files2}")
print(f"Number of files in the folder: {num_files3}")


Code for Contour and Area Analysis

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.spatial import distance
import re
import zipfile
from google.colab import files

# Define constants and paths
PIXELS_TO_MICROMETERS = 9.2667  # Conversion factor
image_dirs = ['/content/images1', '/content/images2', '/content/images3']
thresholded_dir = '/content/thresholded_images'
contour_dir = '/content/contour_images'
excel_dir = '/content/excel_files'
selected_pi_dir = '/content/selectedpi'
output_zip = '/content/processed_results.zip'
selected_pi_excel = '/content/selected_pi_details.xlsx'
sorted_excel_file = '/content/sorted_selected_pi_details.xlsx'
diffusion_file = '/content/Diffusion_coefficients.txt'
final_output_file = '/content/matched_particle_data_with_difference.csv'

# Create necessary directories
os.makedirs(thresholded_dir, exist_ok=True)
os.makedirs(contour_dir, exist_ok=True)
os.makedirs(excel_dir, exist_ok=True)

# Step 1: Process images
def process_and_analyze_image(image_path, thresholded_path, contour_path, excel_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    adaptive_thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Save thresholded image with reduced quality
    cv2.imwrite(thresholded_path, adaptive_thresh, [int(cv2.IMWRITE_JPEG_QUALITY), 30])  # Quality = 30

    # Morphological cleaning
    kernel = np.ones((5, 5), np.uint8)
    cleaned_image = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)

    # Contour detection and analysis
    contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    data = []
    for contour in contours:
        area_pixels = cv2.contourArea(contour)
        if area_pixels > 10:
            area_micrometers = area_pixels / (PIXELS_TO_MICROMETERS**2)
            (x, y), radius_pixels = cv2.minEnclosingCircle(contour)
            radius_micrometers = radius_pixels / PIXELS_TO_MICROMETERS
            data.append({
                "Area (µm²)": area_micrometers,
                "Center X (pixels)": x,
                "Center Y (pixels)": y,
                "Radius (µm)": radius_micrometers
            })

    # Save data to Excel
    pd.DataFrame(data).to_excel(excel_path, index=False)

    # Save image with contours
    result_image = cv2.cvtColor(cleaned_image, cv2.COLOR_GRAY2BGR)
    for row in data:
        center_x = int(row["Center X (pixels)"])
        center_y = int(row["Center Y (pixels)"])
        radius = int(row["Radius (µm)"] * PIXELS_TO_MICROMETERS)
        cv2.circle(result_image, (center_x, center_y), radius, (0, 255, 0), 2)
    cv2.imwrite(contour_path, result_image, [int(cv2.IMWRITE_JPEG_QUALITY), 30])  # Quality = 30

for image_dir in image_dirs:
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            input_path = os.path.join(image_dir, filename)
            thresholded_path = os.path.join(thresholded_dir, f'thresholded_{filename}')
            contour_path = os.path.join(contour_dir, f'contour_{filename}')
            excel_path = os.path.join(excel_dir, f'{os.path.splitext(filename)[0]}_analysis.xlsx')
            process_and_analyze_image(input_path, thresholded_path, contour_path, excel_path)

# Step 2: Process Selected PI files
data = []
for filename in os.listdir(selected_pi_dir):
    if filename.endswith('.txt') and filename.startswith('selected_PI_'):
        match = re.search(r'\d+', filename.split('_')[2])
        if match:
            particle_id = int(match.group())
            pi_file_path = os.path.join(selected_pi_dir, filename)
            pi_data = pd.read_csv(pi_file_path, delim_whitespace=True, header=None, nrows=1)
            x, y, frame = pi_data.iloc[0, :3]
            data.append([particle_id, x, y, frame])

pd.DataFrame(data, columns=["Particle ID", "X", "Y", "Frame"]).to_excel(selected_pi_excel, index=False)

# Step 3: Sort Selected PI data
selected_pi_data = pd.read_excel(selected_pi_excel)
selected_pi_data.sort_values(by="Particle ID").reset_index(drop=True).to_excel(sorted_excel_file, index=False)

# Step 4: Add Diffusion Coefficients
selected_pi_data = pd.read_excel(sorted_excel_file)
diffusion_data = pd.read_csv(diffusion_file, delim_whitespace=True, header=None, names=["Particle ID", "Diffusion Coefficient"])
selected_pi_data["Diffusion Coefficient"] = selected_pi_data["Particle ID"].map(
    diffusion_data.set_index("Particle ID")["Diffusion Coefficient"]
)

# Step 5: Match with Excel analysis
results = []

# Map frame numbers to Excel files from `excel_dir`
frame_to_file = {}
for f in os.listdir(excel_dir):
    match = re.search(r't(\d+)_analysis\.xlsx', f)
    if match:
        frame_number = int(match.group(1))
        frame_to_file[frame_number] = os.path.join(excel_dir, f)

# Process each row in `selected_pi_data` and match with the closest particle
for _, row in selected_pi_data.iterrows():
    particle_id, x, y, frame = row["Particle ID"], row["X"], row["Y"], row["Frame"]
    excel_file = frame_to_file.get(frame)
    if excel_file and os.path.exists(excel_file):
        data = pd.read_excel(excel_file)
        data['Distance'] = data.apply(
            lambda r: distance.euclidean((r["Center X (pixels)"], r["Center Y (pixels)"]), (x, y)), axis=1
        )
        closest = data.loc[data['Distance'].idxmin()]
        results.append([
            particle_id, x, y, frame, closest["Center X (pixels)"], closest["Center Y (pixels)"],
            closest["Area (µm²)"], closest["Radius (µm)"], row["Diffusion Coefficient"], closest["Distance"]
        ])

columns = [
    "Particle ID", "Original X", "Original Y", "Frame",
    "Matched X", "Matched Y", "Area (µm²)", "Radius (µm)",
    "Diffusion Coefficient", "Computed Difference"
]

# Save the matched data to CSV
pd.DataFrame(results, columns=columns).to_csv(final_output_file, index=False)

# Ensure the matched file is visible in the sidebar
print(f"Matched file saved to: {final_output_file}")

# Download the matched file automatically
files.download(final_output_file)

# Optional: Print files in the directory to confirm visibility in the sidebar
print("\nFiles in the /content directory:")
for f in os.listdir('/content'):
    print(f)

# Step 6: Create and Split ZIP
with zipfile.ZipFile(output_zip, 'w') as zipf:
    for folder in [thresholded_dir, contour_dir, excel_dir]:
        for filename in os.listdir(folder):
            filepath = os.path.join(folder, filename)
            zipf.write(filepath, arcname=filepath.replace('/content/', ''))

print("ZIP file created: processed_results.zip")

# Split ZIP into smaller parts (100 MB each)
!split -b 100M /content/processed_results.zip /content/processed_results_part_

# List generated parts
split_parts = [f for f in os.listdir('/content') if f.startswith('processed_results_part_')]
print("Generated parts:")
for part in split_parts:
    print(part)

# Download each part
for part in split_parts:
    files.download(part)

print("All parts are ready for download.")


Adding Slope

In [ ]:
import pandas as pd

# Constants for Stokes-Einstein relation
K_B = 1.38e-23  # Boltzmann constant (J/K)
T = 298  # Temperature in Kelvin
ETA = 0.89e-3  # Viscosity in Pa·s (milliPascal·seconds converted to SI)
MICROMETER_TO_METER = 1e-6  # Conversion from micrometer to meter

# Step 1: Upload files
excel_file = '/content/matched_particle_data_with_difference (5).csv'  # Replace with your Excel file path
text_file = '/content/slope in log scale.txt'  # Replace with your text file path

# Step 2: Try reading the Excel file
try:
    excel_data = pd.read_excel(excel_file, engine='openpyxl')  # Try using openpyxl for .xlsx files
except Exception as e:
    print(f"Error reading Excel file as Excel: {e}")
    # If there's an error with Excel, try reading it as CSV
    try:
        print("Attempting to read as CSV...")
        excel_data = pd.read_csv(excel_file)  # Try reading it as CSV instead
    except Exception as e2:
        print(f"Error reading file as CSV: {e2}")
        raise  # Raise the error if it fails both ways

# Read the text file
text_data = pd.read_csv(text_file, delim_whitespace=True, header=None, names=["Particle ID", "Slope (log scale)"])

# Step 3: Match and filter data by Particle ID
matched_data = pd.merge(excel_data, text_data, on="Particle ID", how="inner")

# Step 4: Calculate Diffusion Coefficients using Stokes-Einstein relation
def calculate_diffusion_coefficient(radius_micrometers):
    radius_meters = radius_micrometers * MICROMETER_TO_METER
    return (K_B * T) / (6 * 3.14159 * ETA * radius_meters)

# Adding the calculated diffusion coefficient
matched_data["Calculated Diffusion (µm²/s)"] = matched_data["Radius (µm)"].apply(calculate_diffusion_coefficient)

# Step 5: Compute the product of Radius and Diffusion Coefficient
matched_data["Radius × Diffusion Coefficient"] = matched_data["Radius (µm)"] * matched_data["Diffusion Coefficient"]

# Step 6: Add the Relative Diffusion Coefficient
matched_data["Relative Diffusion Coefficient (%)"] = (
    (matched_data["Diffusion Coefficient"] - matched_data["Calculated Diffusion (µm²/s)"]) /
    matched_data["Calculated Diffusion (µm²/s)"]
) * 100

# Step 7: Save results to a new Excel file
output_file = '/content/output_matched_data_with_relative_diffusion.xlsx'
matched_data.to_excel(output_file, index=False)

# Step 8: Notify completion
print(f"Processing complete. Results saved to {output_file}")
